In [41]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [42]:
#jupyter nbextension enable –py –sys-prefix gmaps

In [43]:
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Punta Arenas,CL,2021-10-03 00:39:32,-53.1500,-70.9167,42.91,81,75,9.22
1,1,Sorland,NO,2021-10-03 00:40:00,67.6670,12.6934,53.73,81,100,17.02
2,2,Tezu,IN,2021-10-03 00:41:00,27.9167,96.1667,72.88,97,100,3.11
3,3,Padang,ID,2021-10-03 00:40:17,-0.9492,100.3543,75.25,76,88,1.05
4,4,Vegreville,CA,2021-10-03 00:41:01,53.5001,-112.0518,59.70,29,100,5.75


In [44]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [45]:
gmaps.configure(api_key=g_key)

In [46]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []

    # Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp],dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [47]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity=city_data_df["Humidity"]
fig=gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer=gmaps.heatmap_layer(locations,weights=humidity,dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [48]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
cloudiness=city_data_df["Cloudiness"]
fig=gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer=gmaps.heatmap_layer(locations,weights=cloudiness,dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [49]:
# Heatmap of wind speed
locations = city_data_df[["Lat", "Lng"]]
wind=city_data_df["Wind Speed"]
fig=gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer=gmaps.heatmap_layer(locations,weights=wind,dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [62]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [65]:
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.count()

City_ID       182
City          182
Country       182
Date          182
Lat           182
Lng           182
Max Temp      182
Humidity      182
Cloudiness    182
Wind Speed    182
dtype: int64

In [66]:
preferred_cities_df.head(1)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
3,3,Padang,ID,2021-10-03 00:40:17,-0.9492,100.3543,75.25,76,88,1.05


In [97]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Padang,ID,75.25,-0.9492,100.3543,
11,Praia,CV,79.34,14.9215,-23.5087,
18,Aliaga,PH,81.16,15.5167,120.8667,
19,Caravelas,BR,75.47,-17.7125,-39.2481,
22,Salalah,OM,80.69,17.0151,54.0924,
31,Kapaa,US,87.78,22.0752,-159.3190,
34,Umm Lajj,SA,80.10,25.0213,37.2685,
39,Morgan City,US,75.99,29.6994,-91.2068,
41,Hithadhoo,MV,83.30,-0.6000,73.0833,
42,San Quintin,MX,77.11,30.4833,-115.9500,


In [98]:
import requests
import gmaps


In [99]:
params={"radius": 5000,
    "type": "lodging",
    "key": g_key,}

for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotels = requests.get(base_url, params=params).json()
    try:
        hotel_df.loc[index,"Hotel Name"] = hotels["results"][0]["name"]
    except:
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [100]:
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Padang,ID,75.25,-0.9492,100.3543,Mercure Padang
11,Praia,CV,79.34,14.9215,-23.5087,LT Aparthotel
18,Aliaga,PH,81.16,15.5167,120.8667,Jhenie rich ramos
19,Caravelas,BR,75.47,-17.7125,-39.2481,Pousada dos Navegantes
22,Salalah,OM,80.69,17.0151,54.0924,HAMDAN PLAZA HOTEL SALALAH


In [104]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))

,Lat,Lng
3,-0.9492,100.3543
11,14.9215,-23.5087
18,15.5167,120.8667
19,-17.7125,-39.2481
22,17.0151,54.0924
...,...,...
540,11.8091,15.8632
545,2.0371,45.3438
546,-2.7469,-42.8264
547,-1.8283,-44.8683
